In [ ]:
from aiy.vision.inference import CameraInference
from aiy.vision.models import face_detection
from aiy.leds import Leds, Color

from picamera import PiCamera
from IPython.display import Image, display, clear_output

import contextlib

imageDir = "/home/pi/Pictures"
joyTreshhold = 0.5

In [ ]:
with contextlib.ExitStack() as stack:
    leds   = stack.enter_context(Leds())
    camera = stack.enter_context(PiCamera(sensor_mode=4, resolution=(820, 616)))

    print ("Loading model - hold on ..")
    
    # Do inference on VisionBonnet
    with CameraInference(face_detection.model()) as inference:
        try:   
            for result in inference.run():
                leds.update(Leds.rgb_on(Color.RED))
                faces = face_detection.get_faces(result)
                if len(faces) >= 1:
                    clear_output(wait=True)                 
                    leds.update(Leds.rgb_on(Color.GREEN))
                    print("Joy score: ", faces[0].joy_score)
                    
                    if(faces[0].joy_score > joyTreshhold):
                        print("Joy detected - saving picture ..")
                        camera.capture(imageDir+'/faceDetect.jpg')
                        display(Image(filename=imageDir+'/faceDetect.jpg'))
                        break
                        
        except KeyboardInterrupt:
            print("Interrupted ..")
            
    leds.update(Leds.rgb_off())
    print("Done")